# 02: Detect & Trim Runs

Detect run segments and trim them from the continuous data stream.


In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path
import sys
sys.path.append('../../src')

from sledhead_imu.segmentation.detect_trim_runs import detect_run_segments, trim_run_segment
from sledhead_imu.config import INGEST, TRIM

# Load normalized data
data_dir = Path('../data')
ingest_dir = data_dir / '01_ingest_normalize'
trim_dir = data_dir / '02_detect_trim_runs' / 'segments'

# Find normalized data files
normalized_files = list(ingest_dir.glob('*.csv'))
print(f"Found {len(normalized_files)} normalized files")

if normalized_files:
    # Load first file as example
    df = pd.read_csv(normalized_files[0])
    print(f"Data shape: {df.shape}")
    
    # Detect run segments
    segments = detect_run_segments(df, threshold=1.5)
    print(f"Found {len(segments)} run segments")
    
    # Trim first segment
    if segments:
        start_idx, end_idx = segments[0]
        trimmed_segment = trim_run_segment(df, start_idx, end_idx)
        print(f"Trimmed segment shape: {trimmed_segment.shape}")
        print(f"Segment duration: {trimmed_segment['timestamp'].iloc[-1] - trimmed_segment['timestamp'].iloc[0]}")
else:
    print("No normalized data found. Run 01_ingest_normalize.ipynb first.")
